In [5]:
include("../../src/Finite_Vol/pnjl_FV.jl")
using Plots 
using LaTeXStrings
using DataFrames, CSV

In [6]:
T= 300.0
mu_B = 0.0
X0 = [-0.01, -0.01, -0.20, 0.8, 0.8]
phi = X0[1:3]
Phi1 = X0[4]
Phi2 = X0[5]
R = 1.0
a, b ,c = R, R, R

ints_el = get_nodes_el(128, a, b, c)
ints_sph = get_nodes_sph(128, R)

res_el = Omega(phi, Phi1, Phi2, T/hc, mu_B/hc, ints_el)
res_sph = Omega(phi, Phi1, Phi2, T/hc, mu_B/hc, ints_sph)

println("res_el:=", res_el)
println("res_sph:=", res_sph)
println("diff:=", res_el - res_sph)
#sol = Tmu(T/hc, mu_B/hc, X0, ints)

#println("sol:=", sol)

res_el:=-33.57031981340026
res_sph:=-33.57031981400966
diff:=6.093969773246499e-10


In [7]:
Ts= 300.0:-2:50.0
mu_B = 0.0

Rs = [1.0, 2.0, 3.0, 5.0, 7.0, 10.0]
X0 = zeros(5)
data = zeros(length(Ts), 6)
data[:,1] = Ts
for R in Rs
    println("R = $R")
    X0 = [-0.01, -0.01, -0.20, 0.8, 0.8]

    ints_el = get_nodes_sph(256, R)
    for (i, T) in enumerate(Ts)

        X0 = Tmu(T/hc, mu_B/hc, X0, ints_el)
        data[i,2:end] = X0
    end
    df = DataFrame(data, [:T, :phi_u, :phi_d, :phi_s, :Phi1, :Phi2])
    CSV.write("PNJL_FV_R$(R).csv", df)
end


R = 1.0
R = 2.0
R = 3.0
R = 5.0
R = 7.0
R = 10.0


In [22]:
Ts= 300.0:-2:50.0
mu_B = 0.0

phi = X0[1:3]
Phi1 = X0[4]
Phi2 = X0[5]

as = [1.0, 2.0, 3.0, 5.0, 7.0, 10.0]

data = zeros(length(Ts), 6)
data[:,1] = Ts
for a in as
    println("a=", a,"fm")
    X0 = [-0.01, -0.01, -0.20, 0.8, 0.8]
    a, b, c = a, a, 30.0
    ints_el = get_nodes_el(256, a, b, c)
    for (i, T) in enumerate(Ts)

        X0 = Tmu(T/hc, mu_B/hc, X0, ints_el)
        data[i,2:end] = X0
    end
    df = DataFrame(data, [:T, :phi_u, :phi_d, :phi_s, :Phi1, :Phi2])
    CSV.write("el_a=$a.csv", df)
end

a=1.0fm
a=2.0fm
a=3.0fm
a=5.0fm
a=7.0fm
a=10.0fm


In [47]:
include("Ep_test.jl")

total_mean_curvature_spheroid (generic function with 1 method)

In [56]:
R = 5.0
a, b ,c = 4.0, 4.0, 8.0

(4.0, 4.0, 8.0)

In [57]:
res1 = total_mean_curvature_ellipsoid(a, b, c)
res2 = total_mean_curvature_spheroid(a, c)

println("res1:=", res1)
println("res2:=", res2)
println("diff:=", res1 - res2)

res1:=138.75012325352108
res2:=138.7501232535208
diff:=2.8421709430404007e-13


In [58]:
S = 4 * pi * ((a^1.6075 * b^1.6075 + a^1.6075 * c^1.6075 + b^1.6075 * c^1.6075)/3)^(1/1.6075)

343.45030660349687